Get the entities, labels and countries

In [1]:
import rdflib
filename = 'C:\\Users\\aydxng\\Documents\\FilesForExperiment\\NewTaxononmy\\FundRef_v2020-11_20201112_nowrapper_shadow\\FundRef_v2020-11_20201112_nowrapper_shadow.rdf'
g = rdflib.Graph()
g.load(filename)

In [2]:
entity_ids = []
for x in g:
    #If the element is a "Concept", it is an entity.
    #We add the entity ID to the list
    if x[2] == rdflib.term.URIRef('http://www.w3.org/2004/02/skos/core#Concept'):
        entity_ids.append(x[0])

In [3]:
country_info_file = 'C:\\Users\\aydxng\OneDrive - Reed Elsevier Group ICO Reed Elsevier Inc\\Desktop\\ElsevierData\\countryInfo.txt'
def generate_country_dict(country_info_file):
    name = []
    geoname_ids = []
    with open(country_info_file, encoding='utf-8') as f:
        for line in f.readlines():
            if line[0] != '#':
                line = line.split('\t')
                name.append(line[4])
                geoname_ids.append(line[16])
    geoname_ids_dict = dict(zip(geoname_ids,name))
    #Add the geoname id of EU as it does not exist in country_info.txt
    geoname_ids_dict['6695072'] = 'European Union'
    return geoname_ids_dict
geoname_ids_dict = generate_country_dict(country_info_file)

In [4]:
altLabel= rdflib.term.URIRef('http://www.w3.org/2008/05/skos-xl#altLabel')
literalForm= rdflib.term.URIRef('http://www.w3.org/2008/05/skos-xl#literalForm')
prefLabel= rdflib.term.URIRef('http://www.w3.org/2008/05/skos-xl#prefLabel')

In [5]:
entity_dict = dict()
## REP 2
for e_id in entity_ids:
    #Get entity id
    e_id_stripped = e_id.split("/")[-1]
    #Get country
    country = None
    for x in g.objects(subject=e_id,predicate=rdflib.term.URIRef('http://www.elsevier.com/xml/schema/grant/grant-1.2/country')):
        country = geoname_ids_dict[x.split('/')[-2]]
    #Get labels
    all_labels = []
    for x in g.objects(subject=e_id,predicate=prefLabel):
        all_labels.append(str(g.value(subject=rdflib.term.URIRef(x),predicate=literalForm)))
    for x in g.objects(subject=e_id,predicate=altLabel):
        all_labels.append(str(g.value(subject=rdflib.term.URIRef(x),predicate=literalForm)))
    #This will be the value for this entity
    entity_dict[e_id_stripped] = {'Labels':all_labels,'Country':country}

In [6]:
import pickle
with open('entities.pkl','wb') as f:
    pickle.dump(entity_dict,f)